In [1]:
%load_ext autoreload
%autoreload 2

from satpy import Scene, find_files_and_readers
from pyresample import create_area_def
from PyCoxMunk import PyCoxMunk
from datetime import datetime
from netCDF4 import Dataset
import dask.array as da
import xarray as xr
import numpy as np

def _write_gdal(fname, datas):
    from osgeo import gdal
    driver = gdal.GetDriverByName("GTiff")
    shp = datas.shape
    dst_ds = driver.Create(fname,
                           shp[1],
                           shp[0],
                           1,
                           gdal.GDT_Float32)
    dst_ds.GetRasterBand(1).WriteArray(datas)
    del dst_ds
    
def load_wind(fname, var, ecm_scn=None, ftype='nc', dater=datetime(2000, 1, 1, 0, 0, 0)):
    """Load wind datasets from a file
    Inputs:
     - fname: String, the input filename.
     - var: String, name of variable to read
     - ecm_scn: Scene, an existing scene to save data into. If None, a new Scene is created.
     - ftype: String, file type. Currently only netCDF ('nc') is supported.
     - dater: DateTime, time to set as Scene start_time.
    Returns:
      - ecm_scn: Scene, containing 'u10' and 'v10' winds."""
    if ftype != 'nc':
        raise ValueError("Only netCDF winds are supported at present.")
    fid = Dataset(fname, 'r')
    inv = np.array(fid[var]).squeeze()
    lat = np.array(fid['latitude']).squeeze()
    lon = np.array(fid['longitude']).squeeze()
    if np.nanmax(lon) > 180:
        inv = np.roll(inv, np.round(inv.shape[1]/2).astype(int))
        lon = lon - 180.
    fid.close()
    
    if ecm_scn is None:
        ecm_scn = Scene()
    
    area_ext = (np.nanmin(lon), np.nanmin(lat), np.nanmax(lon), np.nanmax(lat))
    targ_area = create_area_def("source_area",
                                "EPSG:4326",
                                area_extent=area_ext,
                                width=inv.shape[1],
                                height=inv.shape[0])


    ecm_scn[var] = xr.DataArray(da.from_array(inv),
                                coords={'y': lat, 'x': lon},
                                attrs={'start_time': dater})

    ecm_scn[var].attrs['area'] = targ_area
    
    return ecm_scn

In [4]:
sev_fname = 'I:/sat_data/SEV/MSG3-SEVI-MSG15-0100-NA-20160326144242.057000000Z-NA.nat'
u10_fname = 'I:/sat_data/SEV/ecmwf-era5_oper_an_sfc_201603261500.10u.nc'
v10_fname = 'I:/sat_data/SEV/ecmwf-era5_oper_an_sfc_201603261500.10v.nc'

bnames = ['VIS006']

scn = Scene([sev_fname], reader='seviri_l1b_native')
scn.load(bnames, upper_right_corner='NE')

In [5]:
ecm_scn = load_wind(u10_fname, 'u10', ecm_scn=None)
ecm_scn = load_wind(v10_fname, 'v10', ecm_scn=ecm_scn)

In [6]:
ecm_scn2 = ecm_scn.resample(scn[bnames[0]].attrs['area'], resampler='gradient_search')

E:\miniconda3\lib\site-packages\pyresample\gradient\__init__.py:56: UserWarning: You are using the Gradient Search Resampler, which is still EXPERIMENTAL.
  warnings.warn("You are using the Gradient Search Resampler, which is still EXPERIMENTAL.")
E:\miniconda3\lib\site-packages\pyproj\crs\crs.py:1282: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
E:\miniconda3\lib\site-packages\pyproj\crs\crs.py:1282: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
E:\miniconda3\lib\site-packages\pyproj\crs\crs.py:1282: UserWarning: You will likely lose important projection

In [7]:
scn['u10'] = ecm_scn2['u10'].copy()
scn['v10'] = ecm_scn2['v10'].copy()

In [9]:
pcm = PyCoxMunk(scn, bnames, angle_names='calc', delete_when_done=False, mask_bad=False)
pcm.setup_wind(scn['u10'], scn['v10'])

C:\Users\Simon\PycharmProjects\PyCoxMunk\PyCoxMunk\src\CM_SceneGeom.py:190: UserWarning: Some solar zenith values out of range. Clipping.
  warnings.warn("Some solar zenith values out of range. Clipping.")
C:\Users\Simon\PycharmProjects\PyCoxMunk\PyCoxMunk\src\CM_SceneGeom.py:200: UserWarning: Some satellite zenith values out of range. Clipping.
  warnings.warn("Some satellite zenith values out of range. Clipping.")
C:\Users\Simon\PycharmProjects\PyCoxMunk\PyCoxMunk\src\CM_SceneGeom.py:210: UserWarning: Some solar azimuth values out of range. Scaling.
  warnings.warn("Some solar azimuth values out of range. Scaling.")


In [10]:
pcm.retr_coxmunk_refl()

In [11]:
pcm.scn.save_datasets(base_dir='I:/', enhance=False, dtype=np.float32)